In [ ]:
!pip install yfinance

import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler

data = yf.download('GOOGL', start='2012-01-01', end='2016-12-31')
open_prices = data[['Open']].values

scaler = MinMaxScaler()
open_prices_scaled = scaler.fit_transform(open_prices)

X_train, y_train = [], []
for i in range(60, len(open_prices_scaled)):
    X_train.append(open_prices_scaled[i-60:i, 0])
    y_train.append(open_prices_scaled[i, 0])
X_train = np.array(X_train)
y_train = np.array(y_train)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

model = tf.keras.Sequential([
    layers.SimpleRNN(50, return_sequences=True, input_shape=(60, 1)),
    layers.SimpleRNN(50),
    layers.Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

test_data = yf.download('GOOGL', start='2017-01-01', end='2017-12-31')
real_stock_price = test_data[['Open']].values

total_data = pd.concat((data['Open'], test_data['Open']), axis=0)
inputs = total_data[-(len(test_data) + 60):].values.reshape(-1, 1)
inputs = scaler.transform(inputs)

X_test = []
for i in range(60, len(inputs)):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

predicted_stock_price = model.predict(X_test)
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

plt.figure(figsize=(10,6))
plt.plot(real_stock_price, color='red', label='Real')
plt.plot(predicted_stock_price, color='blue', label='Predicted')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()
